# Basic `kosmatau3d` model

Here we will examine what are the basic parameters to set in order to establish a simple three-dimensional model. Since we want this example to be easily-relatable, we will attempt to build the `kosmatau3d` version of the famous Hollenbach and Tielens (1999) PDR structure image.

## grid files

Every model must be created from some datafiles, at least to obtain the necessary parameters (mass, density, and far-UV radiation). Our purpose does not require anything complex with varying mass or density, but rather we will assume some constant far-UV radiation from one direction and calculate how the absorption and radiative transfer affects the intensity in each voxel. For that reason we will first create mock grid files in a new directory, then `kosmatau3d` will read the model information from there.

In this example we will use voxels of size 1 pc containing mass $100$ $M_\odot$ and density $10^4$ cm$^{-3}$. Each voxel will also need some velocity information, so we want $v_\mathrm{vox}=0$ km s$^{-1}$ and $\sigma_\mathrm{ens}=1$ km s$^{-1}$. Finally the far-UV radiation (neglecting absorption) will be constant at $10^5$ $\chi_D$. We will see later how accounting for absorption will affect the far-UV penetration.

In [8]:
import numpy as np
from scipy.interpolate import interp1d

x = np.linspace(1, 10, num=10)
y = np.ones_like(x)

fx = interp1d(x, y, fill_value='extrapolate')
fx(-5000)

array(1.)

## setting the model parameters

Now that we have grid files from which `kosmatau3d` will read the voxel properties, we simply need to pass the necessary parameters to the `Model` constructor.

In [ ]:
parameters = {
              # Model information
              'history_path': '',# r'/mnt/hpc_backup/yanitski/projects/pdr/KT3_history',
              'directory': '',
              'folder': 'HT99',
              'x': 10,
              'y': 5,
              'z': 0,
              'modelType': 'block',

              # Model parameters
              'resolution': 1,
              'molecules': 'all',
              # 'molecules': ['C+ 1', 
              #               'C 1', 'C 2', 
              #               'CO 1', 'CO 2', 'CO 3', 'CO 4', 'CO 5', 
              #               'CO 6', 'CO 7', 'CO 8', 
              #               '13C+ 1', 
              #               '13C 1', '13C 2', 
              #               '13CO 1', '13CO 2', '13CO 3', '13CO 4', '13CO 5', 
              #               '13CO 6', '13CO 7', '13CO 8', 
              #               'HCO+ 1', 'HCO+ 2', 'HCO+ 3', 'HCO+ 4', 'HCO+ 5'],
              'dust': 'PAH',
              # 'dust': ['240um', '550um'],
              'clumpMassRange': [[-1, 2]],
              'clumpMassNumber': [4],
              'clumpNmax': [None],
              'clumpLogFUV' : None,
              'interclumpLogFUV' : None,
              'velocityRange': [-10, 10],
              'velocityNumber': 21,

              # Flags
              'suggested_calc': True,
              'dilled': True,

              # Property factors
              'hi_mass_factor': 1,
              'h2_mass_factor': 1,
              'ensemble_mass_factor': [1, 1],
              'fuv_factor': 1,
              'density_factor': 1,
              'interclump_hi_ratio': 1,
              'r_cmz': 0,

              # Logging
              'timed': False,
              'debug': False,
              'verbose': False
              }